In [2]:
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
import glob

/home/asha/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
file_names=glob.glob("./data/train/*.jpg")
file_names.sort()

In [4]:
len(file_names)

10222

In [5]:
y_train=pd.read_csv("./data/labels.csv",delimiter=",")
y_true=y_train['breed']


In [6]:
def breed_to_numbers(y_train):
    unq=np.unique(y_train)
    n=len(unq)
    numbers = list(range(0, n))
    y_dict=dict(zip(unq,numbers))
    y_true=[]
    for label in y_train:
        n=y_dict[label]
        y_true.append(n)
    return y_true

In [7]:
y_true=breed_to_numbers(y_true)

In [8]:
def parse_function(filename, label):
    image_string = tf.read_file(filename)

    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize_images(image, [224,224])
    
    return image, label

In [9]:
batch_size=32

In [10]:
label=tf.one_hot(y_true,120)
dataset=tf.data.Dataset.from_tensor_slices((file_names,label))
dataset=dataset.map(parse_function)
dataset=dataset.batch(batch_size)



In [11]:
iterator = dataset.make_initializable_iterator()
x,y = iterator.get_next()

In [15]:
 
#convolution, relu and max poling for layer 1
conv1=tf.layers.conv2d(x,32,3,strides=1,activation=tf.nn.relu)  
conv1=tf.layers.max_pooling2d(conv1,2,2,padding='SAME')

cov1=tf.layers.batch_normalization(conv1)

    
#convolution, relu and max poling for layer 2
conv2=tf.layers.conv2d(conv1,64,5,strides=1,activation=tf.nn.relu)  
conv2=tf.layers.max_pooling2d(conv2,2,2,padding='SAME')
conv2=tf.layers.batch_normalization(conv2)


conv3=tf.layers.conv2d(conv2,128,5,strides=1,activation=tf.nn.relu)  
conv3=tf.layers.max_pooling2d(conv3,2,2,padding='SAME')

conv3=tf.layers.batch_normalization(conv3)


fc1=tf.contrib.layers.flatten(conv3)
#fully connected layer
fc1=tf.layers.dense(inputs=fc1, units=2048,activation=tf.nn.relu)
fc2=tf.layers.dense(inputs=fc1, units=512,activation=tf.nn.relu)
logits=tf.layers.dense(inputs=fc2, units=120)
#softmax for probabilities
#prob=tf.nn.softmax(logits)
    
#calculating loss
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train_op = optimizer.minimize(loss_op)
    
#calculating accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init=tf.global_variables_initializer()

In [16]:
batches=len(file_names)/batch_size
batches=int(batches)
epochs=10
print batches

319


In [ ]:

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        
        print "Epoch:",epoch ,"running"
        sess.run(iterator.initializer)
        for batch in range(batches):
            trainop,loss,acc=sess.run([train_op,loss_op,accuracy])
            if batch%10==0:
                print "batch",batch,"loss:",loss,"accuracy",acc
                 
                
        print "Epoch:",epoch,"ended"
        
        


Epoch: 0 running
batch 0 loss: 4.7954416 accuracy 0.0
batch 10 loss: 4.805768 accuracy 0.0
batch 20 loss: 4.787804 accuracy 0.03125
batch 30 loss: 4.801465 accuracy 0.0
batch 40 loss: 4.80332 accuracy 0.0
batch 50 loss: 4.792467 accuracy 0.0
batch 60 loss: 4.7799277 accuracy 0.0
batch 70 loss: 4.7796984 accuracy 0.0625
batch 80 loss: 4.783596 accuracy 0.0
batch 90 loss: 4.769043 accuracy 0.03125
batch 100 loss: 4.767809 accuracy 0.0
batch 110 loss: 4.7925386 accuracy 0.0625
batch 120 loss: 4.7974877 accuracy 0.03125
batch 130 loss: 4.7799916 accuracy 0.0
batch 140 loss: 4.692598 accuracy 0.03125
batch 150 loss: 4.798873 accuracy 0.03125
batch 160 loss: 4.6637325 accuracy 0.0
batch 170 loss: 4.635807 accuracy 0.0625
batch 180 loss: 4.770919 accuracy 0.0
batch 190 loss: 4.846982 accuracy 0.0
batch 200 loss: 4.7438297 accuracy 0.03125
batch 210 loss: 4.7422733 accuracy 0.03125
batch 220 loss: 4.6512365 accuracy 0.03125
batch 230 loss: 4.6019077 accuracy 0.09375
batch 240 loss: 4.8059177 a